In [1]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd

from scipy.io import arff
data = arff.loadarff('jura.arff')
df = pd.DataFrame(data[0])

covariables = df.iloc[:,0:15].values
response = df.iloc[:,15:18].values
positions = np.arange(359)

from sklearn.model_selection import train_test_split


covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


##APPRENTISSAGE

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = numpy.empty(a.shape, dtype=a.dtype)
    shuffled_b = numpy.empty(b.shape, dtype=b.dtype)
    permutation = numpy.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

#import tensorflow as tf 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()



# Import data
# Tensorflow is finicky about shapes, so resize


X_datatrain, X_datatest, targets_train, targets_test = covariables_train, covariables_test, response_train, response_test

#les dimensions de X for training
NbrLignes_train = X_datatrain.shape[0]  ###
NbrColonnes_train= X_datatrain.shape[1] ###

#les dimensions de X for testing
NbrLignes_test = X_datatest.shape[0]  ###
NbrColonnes_test = X_datatest.shape[1] ###
#Y_data = targets 

Y_datatrain = tf.reshape(targets_train, [NbrLignes_train,targets_train.shape[1]])
Y_datatest = tf.reshape(targets_test, [NbrLignes_test,targets_train.shape[1]])

#build the model

# input X: batch_size x NbrColonnes, the first dimension (None) will index the data in the mini-batch
Xfill = tf.placeholder(tf.float32, shape= [None, NbrColonnes_train])
# correct answers will go here
Yfill = tf.placeholder(tf.float32, shape=[None, targets_train.shape[1]]) 
#Let's add some layers

# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 10 #200
M = 3 #160
#N = 100
#O = 30

# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([NbrColonnes_train, L], stddev=0.1))  # 
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
#W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
#B3 = tf.Variable(tf.ones([N])/10)
#W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
#B4 = tf.Variable(tf.ones([O])/10)
#W5 = tf.Variable(tf.truncated_normal([O, 6], stddev=0.1))
#B5 = tf.Variable(tf.zeros([6]))

# The model, with dropout at each layer


Y1 = tf.nn.relu(tf.matmul(Xfill, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

#Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
#Y2d = tf.nn.dropout(Y2, pkeep)

#Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
#Y3d = tf.nn.dropout(Y3, pkeep)

#Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
#Y4d = tf.nn.dropout(Y4, pkeep)

#Y_ = tf.matmul(Y4d, W5) + B5


Y_ = tf.matmul(Y1d, W2) + B2

cross_entropy = tf.losses.mean_squared_error(Yfill, Y_)


# accuracy of the trained model, between 0 (worst) and 1 (best)
#Modification de l'accuracy 

#calcul de l'accuracy, une nouvelle manière: :  

# variable learning rate
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy) 

# init
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 100.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

train_loss = []
test_loss = []

# Gradient descent loop for 500 steps
for i in range(3500):
 # Select random minibatch
    j = 1
    ##### shuffling our data::: really important
    X_datasf, Y_datasf = shuffle_in_unison(X_datatrain, Y_datatrain.eval())
    
    ##### defining a decreasing learning rate: 
    # learning rate decay
    learning_rate = 0.0001001329829992624
    #learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    ##### 
    for start, end in zip(range(0, NbrLignes_train, 52), range(20, NbrLignes_train, 52)):

             X_batch,Y_batch = X_datasf[start:end,],Y_datasf[start:end,]
             j +=1
             # Do gradient descent step
             _, loss_val_train = sess.run([train_step, cross_entropy], feed_dict={Xfill: X_batch, Yfill: Y_batch, pkeep: 0.75, lr: learning_rate})
             if i %20==0:    
                 
             
                #if j%20==0:
                 loss_val_test = sess.run(cross_entropy, feed_dict={Xfill: X_datatest, Yfill: Y_datatest.eval(), pkeep: 1.0, lr:  learning_rate})
                 train_loss.append(loss_val_train)
                 
                 test_loss.append(loss_val_test)
                 print('LOSS at step %s: - Train loss value : %f -- Test loss value : %f' % (i, loss_val_train,loss_val_test))
#print(test_loss)
#print(test_loss)
#LOSS at step 420: - Train loss value : 22.385496 -- Test loss value : 9.721767

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
LOSS at step 0: - Train loss value : 195.957367 -- Test loss value : 287.872772
LOSS at step 0: - Train loss value : 508.547241 -- Test loss value : 287.635498
LOSS at step 0: - Train loss value : 391.923126 -- Test loss value : 287.401794
LOSS at step 0: - Train loss value : 473.307281 -- Test loss value : 287.167938
LOSS at step 0: - Train loss value : 337.889008 -- Test loss value : 286.933624
LOSS at step 0: - Train loss value : 818.210571 -- Test loss value : 286.700928
LOSS at step 20: - Train loss value : 500.521667 -- Test loss value : 262.503204
LOSS at step 20: - Train loss value : 215.111877 -- Test loss value : 262.311066
LOSS at step 20: - Train loss value : 391.285919 -- Test loss value : 262.123779
LOSS at step 20: - Train loss value : 333.682617 -- Test loss value : 261.935

LOSS at step 320: - Train loss value : 88.416603 -- Test loss value : 33.263580
LOSS at step 320: - Train loss value : 101.440208 -- Test loss value : 33.256721
LOSS at step 320: - Train loss value : 108.616631 -- Test loss value : 33.249496
LOSS at step 320: - Train loss value : 37.817707 -- Test loss value : 33.240932
LOSS at step 320: - Train loss value : 88.838219 -- Test loss value : 33.234123
LOSS at step 320: - Train loss value : 112.886826 -- Test loss value : 33.226917
LOSS at step 340: - Train loss value : 97.915314 -- Test loss value : 32.697041
LOSS at step 340: - Train loss value : 88.853134 -- Test loss value : 32.691406
LOSS at step 340: - Train loss value : 57.930183 -- Test loss value : 32.684387
LOSS at step 340: - Train loss value : 48.585758 -- Test loss value : 32.675541
LOSS at step 340: - Train loss value : 160.228745 -- Test loss value : 32.671013
LOSS at step 340: - Train loss value : 92.681244 -- Test loss value : 32.668736
LOSS at step 360: - Train loss value

LOSS at step 660: - Train loss value : 136.901077 -- Test loss value : 23.279360
LOSS at step 660: - Train loss value : 35.887096 -- Test loss value : 23.277584
LOSS at step 660: - Train loss value : 38.447948 -- Test loss value : 23.277100
LOSS at step 660: - Train loss value : 51.957607 -- Test loss value : 23.272707
LOSS at step 660: - Train loss value : 160.713165 -- Test loss value : 23.271692
LOSS at step 660: - Train loss value : 61.099205 -- Test loss value : 23.271059
LOSS at step 680: - Train loss value : 174.291779 -- Test loss value : 22.949678
LOSS at step 680: - Train loss value : 277.834930 -- Test loss value : 22.959963
LOSS at step 680: - Train loss value : 65.192223 -- Test loss value : 22.966150
LOSS at step 680: - Train loss value : 117.582840 -- Test loss value : 22.974022
LOSS at step 680: - Train loss value : 29.691568 -- Test loss value : 22.978836
LOSS at step 680: - Train loss value : 107.827217 -- Test loss value : 22.987228
LOSS at step 700: - Train loss val

LOSS at step 1040: - Train loss value : 34.971672 -- Test loss value : 21.653061
LOSS at step 1040: - Train loss value : 24.977713 -- Test loss value : 21.642179
LOSS at step 1040: - Train loss value : 88.504898 -- Test loss value : 21.636162
LOSS at step 1040: - Train loss value : 52.273323 -- Test loss value : 21.621960
LOSS at step 1040: - Train loss value : 50.949192 -- Test loss value : 21.612822
LOSS at step 1040: - Train loss value : 102.690056 -- Test loss value : 21.605101
LOSS at step 1060: - Train loss value : 50.125065 -- Test loss value : 21.256325
LOSS at step 1060: - Train loss value : 71.657860 -- Test loss value : 21.245251
LOSS at step 1060: - Train loss value : 412.369324 -- Test loss value : 21.248655
LOSS at step 1060: - Train loss value : 38.711586 -- Test loss value : 21.253675
LOSS at step 1060: - Train loss value : 47.372593 -- Test loss value : 21.249268
LOSS at step 1060: - Train loss value : 76.266037 -- Test loss value : 21.244169
LOSS at step 1080: - Train

LOSS at step 1380: - Train loss value : 78.443085 -- Test loss value : 21.341173
LOSS at step 1380: - Train loss value : 68.735855 -- Test loss value : 21.352800
LOSS at step 1380: - Train loss value : 202.602570 -- Test loss value : 21.366386
LOSS at step 1380: - Train loss value : 31.185427 -- Test loss value : 21.378733
LOSS at step 1380: - Train loss value : 291.037109 -- Test loss value : 21.401579
LOSS at step 1380: - Train loss value : 91.799355 -- Test loss value : 21.425167
LOSS at step 1400: - Train loss value : 19.786900 -- Test loss value : 21.885208
LOSS at step 1400: - Train loss value : 34.422867 -- Test loss value : 21.884403
LOSS at step 1400: - Train loss value : 98.631111 -- Test loss value : 21.881502
LOSS at step 1400: - Train loss value : 93.741249 -- Test loss value : 21.878613
LOSS at step 1400: - Train loss value : 51.571812 -- Test loss value : 21.874758
LOSS at step 1400: - Train loss value : 24.752323 -- Test loss value : 21.869734
LOSS at step 1420: - Train

LOSS at step 1720: - Train loss value : 70.047928 -- Test loss value : 21.936922
LOSS at step 1720: - Train loss value : 194.488785 -- Test loss value : 21.924585
LOSS at step 1720: - Train loss value : 78.865875 -- Test loss value : 21.916307
LOSS at step 1720: - Train loss value : 37.282597 -- Test loss value : 21.910036
LOSS at step 1720: - Train loss value : 180.678467 -- Test loss value : 21.906906
LOSS at step 1720: - Train loss value : 78.666389 -- Test loss value : 21.907898
LOSS at step 1740: - Train loss value : 42.948135 -- Test loss value : 21.960611
LOSS at step 1740: - Train loss value : 47.926434 -- Test loss value : 21.957369
LOSS at step 1740: - Train loss value : 86.165451 -- Test loss value : 21.947556
LOSS at step 1740: - Train loss value : 18.842350 -- Test loss value : 21.935949
LOSS at step 1740: - Train loss value : 83.325562 -- Test loss value : 21.930605
LOSS at step 1740: - Train loss value : 146.494156 -- Test loss value : 21.931822
LOSS at step 1760: - Trai

LOSS at step 2080: - Train loss value : 28.881908 -- Test loss value : 22.154509
LOSS at step 2080: - Train loss value : 151.926086 -- Test loss value : 22.172401
LOSS at step 2080: - Train loss value : 28.609589 -- Test loss value : 22.182753
LOSS at step 2080: - Train loss value : 30.962442 -- Test loss value : 22.190357
LOSS at step 2080: - Train loss value : 41.366413 -- Test loss value : 22.190235
LOSS at step 2080: - Train loss value : 106.052872 -- Test loss value : 22.188162
LOSS at step 2100: - Train loss value : 54.695686 -- Test loss value : 22.042177
LOSS at step 2100: - Train loss value : 54.643547 -- Test loss value : 22.049435
LOSS at step 2100: - Train loss value : 15.592887 -- Test loss value : 22.055071
LOSS at step 2100: - Train loss value : 35.355198 -- Test loss value : 22.061375
LOSS at step 2100: - Train loss value : 23.682779 -- Test loss value : 22.064939
LOSS at step 2100: - Train loss value : 25.377750 -- Test loss value : 22.069336
LOSS at step 2120: - Train

LOSS at step 2440: - Train loss value : 96.471748 -- Test loss value : 21.182999
LOSS at step 2440: - Train loss value : 109.995399 -- Test loss value : 21.176252
LOSS at step 2440: - Train loss value : 104.495285 -- Test loss value : 21.175686
LOSS at step 2440: - Train loss value : 22.920399 -- Test loss value : 21.173233
LOSS at step 2440: - Train loss value : 30.521345 -- Test loss value : 21.169176
LOSS at step 2440: - Train loss value : 133.318466 -- Test loss value : 21.172163
LOSS at step 2460: - Train loss value : 363.355743 -- Test loss value : 21.203352
LOSS at step 2460: - Train loss value : 141.651855 -- Test loss value : 21.220812
LOSS at step 2460: - Train loss value : 32.126820 -- Test loss value : 21.237350
LOSS at step 2460: - Train loss value : 17.667957 -- Test loss value : 21.249247
LOSS at step 2460: - Train loss value : 20.062426 -- Test loss value : 21.258175
LOSS at step 2460: - Train loss value : 87.010735 -- Test loss value : 21.272804
LOSS at step 2480: - Tr

LOSS at step 2800: - Train loss value : 52.208332 -- Test loss value : 21.034359
LOSS at step 2800: - Train loss value : 21.104187 -- Test loss value : 21.049452
LOSS at step 2800: - Train loss value : 37.368969 -- Test loss value : 21.066637
LOSS at step 2800: - Train loss value : 132.136627 -- Test loss value : 21.083036
LOSS at step 2800: - Train loss value : 56.413906 -- Test loss value : 21.094397
LOSS at step 2800: - Train loss value : 146.341873 -- Test loss value : 21.109701
LOSS at step 2820: - Train loss value : 48.012989 -- Test loss value : 21.385660
LOSS at step 2820: - Train loss value : 27.656584 -- Test loss value : 21.374056
LOSS at step 2820: - Train loss value : 59.445564 -- Test loss value : 21.361771
LOSS at step 2820: - Train loss value : 44.918106 -- Test loss value : 21.346106
LOSS at step 2820: - Train loss value : 62.289326 -- Test loss value : 21.335146
LOSS at step 2820: - Train loss value : 56.580212 -- Test loss value : 21.326969
LOSS at step 2840: - Train

LOSS at step 3160: - Train loss value : 32.875839 -- Test loss value : 21.021521
LOSS at step 3160: - Train loss value : 28.871387 -- Test loss value : 21.020424
LOSS at step 3160: - Train loss value : 30.842607 -- Test loss value : 21.015654
LOSS at step 3160: - Train loss value : 162.538605 -- Test loss value : 21.015734
LOSS at step 3160: - Train loss value : 65.940071 -- Test loss value : 21.018528
LOSS at step 3160: - Train loss value : 25.548838 -- Test loss value : 21.021696
LOSS at step 3180: - Train loss value : 113.406853 -- Test loss value : 21.179468
LOSS at step 3180: - Train loss value : 104.064323 -- Test loss value : 21.176804
LOSS at step 3180: - Train loss value : 118.667084 -- Test loss value : 21.171936
LOSS at step 3180: - Train loss value : 31.840431 -- Test loss value : 21.165831
LOSS at step 3180: - Train loss value : 22.886312 -- Test loss value : 21.158081
LOSS at step 3180: - Train loss value : 133.197403 -- Test loss value : 21.150496
LOSS at step 3200: - Tr

# Let us compute the aRRMSE

In [2]:
learning_rate = 0.0001001329829992624

Y_testpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatest, tf.float32).eval(), Yfill: tf.cast(Y_datatest, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})
#numpy.savetxt('Y_testpred_with_stock_%s_and_ai_%s_modelcut_%s_vartimeframe_%s.txt'%(stock, ai, cut, vartimeframe), Y_testpred, fmt='%f')
Y_trainpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatrain, tf.float32).eval(), Yfill: tf.cast(Y_datatrain, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})


In [3]:
Y_testpred

array([[  1.429879  ,   9.628322  ,  20.408043  ],
       [  1.4369678 ,   9.584709  ,  14.68537   ],
       [  1.214452  ,   7.7781043 ,  16.83396   ],
       [  0.8659981 ,   4.958919  ,  25.647192  ],
       [  1.4149506 ,   9.634891  ,  17.149965  ],
       [  1.2009435 ,   8.412229  ,   6.033302  ],
       [  0.9325594 ,   5.476868  ,  15.014771  ],
       [  1.4646864 ,   9.793988  ,  21.614725  ],
       [  1.6194739 ,  11.354625  ,  19.145811  ],
       [  1.6010108 ,  10.428918  ,  54.615643  ],
       [  1.0481856 ,   6.6778355 ,  20.428717  ],
       [  1.3199515 ,   8.848412  ,  12.427581  ],
       [  1.3499949 ,   9.130618  ,  15.789906  ],
       [  1.323465  ,   8.816404  ,  15.739203  ],
       [  1.3044655 ,   8.346617  ,  20.019455  ],
       [  1.6726367 ,  11.539459  ,  25.43705   ],
       [  1.6656537 ,  12.267958  ,  16.14478   ],
       [  1.3416061 ,   8.417782  ,  43.801525  ],
       [  1.2394555 ,   8.271538  ,  32.910114  ],
       [  1.1847138 ,   7.88306

In [4]:
from numpy import linalg as LA

In [5]:

aRRMSE_list = []

for i in range(Y_testpred.shape[1]) :
    
    Errors_test =  Y_testpred[:,i] - response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    ytrain_mean_i = np.tile(Y_trainpred[:,i].mean(axis = 0), (covariables_test.shape[0], 1))

    Errors_relative = ytrain_mean_i - response_test[:,i]

    Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%Final_SPNNR_aRRMSE)

    aRRMSE_list.append(Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 8.828631
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 9.468846
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 5.081301


In [6]:
np.mean(aRRMSE_list)

7.792925902342339

In [9]:
#Taking aRRMSE on PCA or inverted PCA. ?

# Check if old way calculation is wrong

In [10]:
Errors_test =  Y_testpred - response_test


print('<*********************************Let us check the aRRMSE*********************************>')

ytrain_mean = np.tile(Y_trainpred.mean(axis = 0), (response_test.shape[0], 1))

Errors_relative = ytrain_mean - response_test

Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

print("Final_Partition aRRMSE is : %f"%Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final_Partition aRRMSE is : 82.647642


*** The answer is yes ***

# Checking True aRRMSE of S-SPNNR on Slump

In [11]:
spnnr_covariables_train, spnnr_covariables_test, spnnr_response_train, spnnr_response_test,spnnr_positions_train,spnnr_positions_test = train_test_split(covariables, response,positions, test_size=0.33, random_state=42)

In [12]:
spnnr_Y_testpred =np.loadtxt('All_Y_test_predictions.out', delimiter=',' )
spnnr_Y_trainpred =np.loadtxt('All_Y_train_predictions.out', delimiter=',' )

In [10]:
spnnr_aRRMSE_list = []

for i in range(spnnr_Y_testpred.shape[1]) :
    
    spnnr_Errors_test =  spnnr_Y_testpred[:,i] - spnnr_response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    spnnr_ytrain_mean_i = np.tile(spnnr_Y_trainpred[:,i].mean(axis = 0), (spnnr_covariables_test.shape[0], 1))

    spnnr_Errors_relative = spnnr_ytrain_mean_i - spnnr_response_test[:,i]

    spnnr_Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(spnnr_Errors_test)**2)/(LA.norm(spnnr_Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%spnnr_Final_SPNNR_aRRMSE)

    spnnr_aRRMSE_list.append(spnnr_Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 9.212108
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 17.784301
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.862991


In [11]:
np.mean(spnnr_aRRMSE_list)

15.286466834678842